In [ ]:
from mantid.simpleapi import *
from mantid.api import MatrixWorkspace
from reduction_workflow.command_interface import ReductionSingleton, Clear
from mantid.api import *
from mantid.kernel import *
from sansdata import *

sans_file = SansData("data/memb_BS_Q1_6_0Ang.mpa")
sans_file.plot_2d()

In [ ]:
from reduction_workflow.command_interface import *
nBins = 1

delta_L_over_L0 = 0.1
bin_lower = sans_file.L0 * (1.0 - delta_L_over_L0/2)
bin_upper = sans_file.L0 * (1.0 + delta_L_over_L0/2)

detectors = 1024 * 1024

# Define single wavelength bin
x = np.repeat([bin_lower,bin_upper],detectors)
# Counts
y = sans_file.raw_intensity
# The workspace will be named "dataWS1", error values will be zero.
ws = CreateWorkspace(OutputWorkspace = "raw_2D_counts",UnitX="Wavelength",DataX=x, DataY=y, NSpec=detectors)
mon = LoadInstrument(ws, FileName="RIDSANS_Definition.xml", RewriteSpectraMap=True)
# SetSampleDetectorDistance(1802.5)
# SetWavelength(4.86, 0.13)
inst = ws.getInstrument()
di2 = ws.detectorInfo()
ci2 = ws.componentInfo()
print("Workspace {0} has instrument: {1}".format(ws.name(), inst.getName()))
print("Instrument {0} has {1} components, including {2} monitors and {3} detectors".format(inst.getName(), ci2.size(), len(mon), di2.size()))

In [27]:
# print("number of histograms = {0}".format(ws.getNumberHistograms()))
# # ws.getSpectrum(3)
# for i in range(0,ws.getNumberHistograms(),1000):
#     y = ws.readY(i)
#     sum_counts = 0
#     # Inner loop. Loop over bins.
#     for j in range(ws.blocksize()):
#         sum_counts += y[j] 
#     # Display spectrum number against sum_counts
#     if sum_counts != 0.0:
#         print("{0} {1}".format(ws.getSpectrum(i).getSpectrumNo(), sum_counts))

In [ ]:
delta_Q = 1e-11
max_QXY = 1e-8
Qxy(ws,MaxQxy=max_QXY, DeltaQ=delta_Q,OUTPUTWORKSPACE="new")

In [ ]:
from mantid.api import AnalysisDataService as ADS
from mantid.plots.utility import MantidAxType
import numpy as np
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogLocator
from asciiexport import *

reduced_ws = ADS.retrieve('new')
print(reduced_ws)
axis = reduced_ws.getAxis(1).extractValues()
print(axis)

for index in range(0, reduced_ws.getNumberHistograms()):
    #Note the round brackets followed by the square brackets
    print(reduced_ws.readX(index)[0])
# y_data2 = ws.readX(1)
# for y in y_data2:
#     print(y)